<a href="https://colab.research.google.com/github/Alex64-1149/VoxNote/blob/IA-VoxNote/VoxNote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adaptation des données

In [ ]:
#Alphabet dans : https://github.com/dallal9/French-NLP

In [ ]:
#Initialisation des caractères possibles
alphabet = [' ','a', 'à', 'A', 'Á', 'À', 'Â', 'Ä', 'Ã', 'Å', 'r', 'd', 'v', 'k', 's', 'b', 'i', 'e', 'n', 't', 'â', 'm', 'z', 'o', 'é', 'è', 'q', 'u', 'î', '-', 'j', 'g', 'l', 'x', 'y', 'c', 'ê', 'h', 'f', 'p', 'B', 'S', 'û', 'ç', 'ô', 'X', 'ï', 'D', 'N', 'Æ', 'G', 'T', 'C', 'ë', 'K', 'L', 'w', 'ö', 'P', 'I', 'R', "'", 'Z', 'E', 'H', 'Ç', 'O', 'M', 'É', 'U', 'V', 'Ð', 'ü', 'È', 'Ê', 'Ë', 'F', 'Q', '3', 'W', 'Í', 'Ì', 'Î', 'Ï', 'J', 'Y', '/', 'Ñ', 'Ó', 'Ò', 'Ô', 'Ö', 'Õ', 'Ø', 'ù', 'þ', 'Ú', 'Ù', 'Û', 'Ü', 'Ý']
chiffres = ['1','2','3','4','5','6','7','8','9','0']
caracteresPossible = alphabet + chiffres


In [8]:

#adaptation de https://github.com/musikalkemist/pytorchforaudio

import torch
import torchaudio
from torch.utils.data import Dataset
import pandas as pd


class ReconnaissanceVocaleDataset(Dataset) :

  #Initialise les aurguments de la classe dataset:
  # 1.fichier annotation(fichier txt qui contient tous les noms des fichiers audio (wav))
  # 2.fichier audio ( contient tous les fichiers audio(wav))
  def __init__(self,FICHIER_ANNOTE,FICHIER_AUDIO, melSpectrogram, SAMPLE_RATE, NOMBRE_ECHANTILLONS, processeur):
    self.fichierAnnotations=pd.read_csv(FICHIER_ANNOTE)
    self.fichierAudio=FICHIER_AUDIO
    self.processeur = processeur
    self.melSpectrogram = melSpectrogram.to(processeur) #s'assurer que tout se fasse au même endroit dans l'ordinateur : https://stackoverflow.com/questions/63061779/pytorch-when-do-i-need-to-use-todevice-on-a-model-or-tensor
    self.SAMPLE_RATE_VOULU = SAMPLE_RATE
    self.NOMBRE_ECHANTILLONS = NOMBRE_ECHANTILLONS

  #retourne le nombre de fichiers dans notre dataset
  def __len__(self):
    return len(self.fichierAnnotations)

  #retourne l'audio ainsi que son fichier texte associé
  def __getitem__(self, index):
    pathAudio = self.getAudioSamplePath(index)
    texte = self.getAudioSampleText(index)
    #signal informatique et sample rate de notre audio
    signal, sr = torchaudio.load(pathAudio)

    #mettre le signal au même endroit que sa transformation
    signal = signal.to(processeur)

    #normaliser l'audio
    signal = self.resampleSiNecessaire(signal, sr)
    signal = self.combinerSiNecessaire(signal)
    #diminuer ou ajouter des échantillons "vides" si le nombre d'échantillons ne correspond à 22 050
    signal = self.couperSiNecessaire(signal)
    signal = self.paddingSiNecessaire(signal)

    #transformer l'audio dans le spectogram de mel
    signal = self.melSpectrogram(signal)
    return signal, texte

  #mettre tous les fichiers audios à la même fréquence d'échantillonage
  def resampleSiNecessaire(self, signal, sr):
    if sr != self.SAMPLE_RATE_VOULU :
      resampler = torchaudio.transforms.Resample(sr, self.SAMPLE_RATE_VOULU)
      signal = resampler(signal)
    return signal

  #s'assurer que l'audio ne contient qu'une entrée et sortie (que le son ne soit pas stéréo) pour le normaliser
  def combinerSiNecessaire(self, signal):
    if signal.shape[0] > 1 :
      signal = torch.mean(signal, dim=0, keepdim= True)
    return signal

  #enlever les échantillons audios superflus
  def couperSiNecessaire(self, signal) :
    #le signal est composé de 2 dimensions : [nombre de source du signal(1 dans ce cas), longueur du signal – nombre d'échantillons (on veut 22 050)]
    if signal.shape[1] > self.NOMBRE_ECHANTILLONS :
      signal = signal[:, :self.NOMBRE_ECHANTILLONS] #utilité de [:, :] : la première dimension est prise au complet et la deuxième jusqu'à l'atteinte du nombre d'échantillon (expliquer à https://youtu.be/WyJvrzVNkOc?list=PL-wATfeyAMNoirN4idjev6aRu8ISZYVWm&t=478)
    return signal

  def paddingSiNecessaire(self, signal) :
    #la longueur du signal = signal.shape[1] comme expliqué précédemment
    if signal.shape[1] < self.NOMBRE_ECHANTILLONS :
      paddingDuSignal = (0, self.NOMBRE_ECHANTILLONS - signal.shape[1]) #(0, nombre d'échantillons manquants)
      torch.nn.functional.pad(signal, paddingDuSignal) #ajoute le padding au signal (https://pytorch.org/docs/stable/generated/torch.nn.functional.pad.html)
    return signal


  #retourne le chemin pour avoir le bon fichier audio à un certain index du FICHIER_ANNOTE
  def getAudioSamplePath(self, index):
    return self.fichierAudio[index]

  #retourne le fichier texte associé à l'audio d'un certain index du FICHIER_AUDIO
  def getAudioSampleText(self, index):
    return self.fichierAnnotations[index]


FICHIER_ANNOTE = "/content/drive/MyDrive/Colab Notebooks/SiwisFrenchSpeechSynthesisDatabase/lists/all_text.list"
FICHIER_AUDIO = "/content/drive/MyDrive/Colab Notebooks/SiwisFrenchSpeechSynthesisDatabase/lists/all_wavs.list"


#nombre d'échantillons par secondes dans notre audio
SAMPLE_RATE = 22050
NOMBRE_ECHANTILLONS = 22050

if torch.cuda.is_available(): #détermine ce qui exécute le programme (gpu préférable pour AI audio)
    processeur = "cuda"
else:
    processeur = "cpu"
print(f"Utiliation du processeur {processeur}")

#le Spectogram de Mel est une échelle logarithmique utilisée pour mieux représenter les différences qu'un humain entend dans un fichier audio ce qui aide à l'analyse sonore
melSpectrogram = torchaudio.transforms.MelSpectrogram(
    SAMPLE_RATE,
    n_fft=512, #longueur physique du signal optimale pour la reconnaissance vocale selon : https://librosa.org/doc/main/generated/librosa.stft.html
    hop_length=512, #nombre d'échantillon audio adjacents analysés par la transformée de fourier : https://librosa.org/doc/main/generated/librosa.stft.html
    n_mels=64 #nombre de séparations d'une seule fréquence optimale pour la reconnaisance vocale selon:https://stackoverflow.com/questions/62623975/why-128-mel-bands-are-used-in-mel-spectrograms
)


rvd = ReconnaissanceVocaleDataset(FICHIER_ANNOTE, FICHIER_AUDIO, melSpectrogram, SAMPLE_RATE, NOMBRE_ECHANTILLONS, processeur)


Utiliation du cuda


In [7]:
from google.colab import drive #nécessaire qu début de chaque session
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#utilisé pour extraire le zip de google drive
#from google.colab import drive
#drive.mount('/content/drive')


#!unzip file.zip


# Réseau de neurone

# Entraîner

# Tester


# Inférence